# Explore rechunked ERA5 

In [ ]:
import zarr
import fsspec
from dotenv import load_dotenv
import xarray as xr
import dask
from dask.distributed import LocalCluster, Client

In [ ]:
import os
if 'NEBARI_JUPYTERHUB_SSH_PORT' in os.environ.keys():   # NEBARI
    dot_env_file = '/shared/users/nebari-setup/lib/CMRE_Blob_Keys.env'
elif 'AML_CloudName' in os.environ.keys():  # AzureML
    dot_env_file = '/home/azureuser/cloudfiles/code/Users/CMRE_Blob_Keys.env'
else:
    print('Unknown Compute Environment: Blob_Keys not found')
print(dot_env_file)
_ = load_dotenv(dot_env_file)  # loads Azure credentials

In [ ]:
storage_options={
            "account_name": os.environ["AZURE_STORAGE_CONNECTION_STRING"],
            "account_key": os.environ["AZURE_STORAGE_KEY"],
        }

In [ ]:
fs = fsspec.filesystem('abfs', storage_options=storage_options, use_listings_cache=False, skip_instance_cache=True)

In [ ]:
#region = 'usa'
region = 'europe'
combined_parquet_abfs = f'abfs://cmre-cc01/data/era5/combined_nc_{region}.parq'

In [ ]:
opts = storage_options
so=dict(remote_protocol='abfs', remote_options=opts, target_options=opts)

In [ ]:
cluster = LocalCluster(threads_per_worker=1)
client = Client(cluster)

In [ ]:
ds = xr.open_dataset(combined_parquet_abfs, engine="kerchunk", chunks={},
            storage_options=dict(remote_protocol='abfs', remote_options=opts, target_options=opts, lazy=True))

In [ ]:
ds

In [ ]:
%%time
da = ds['2m_temperature'][:,100,200].load()

In [ ]:
da

In [ ]:
da.plot()